# oc case study

This case study concerns the transient difference absorption measured in the visible spectral region after 530 nm excitation of an articial light harvester, named orange-calix (oc), in the solvent toluene. The measurement, the global analysis and the target analysis is based upon the model explained in ([Hippius et al 2007][Hippius2007]).

[Hippius2007]: https://www.few.vu.nl/~ivo/pub/2007/HippiusWilliamsWuerthnerJPCC111_13988_13996.pdf "C. Hippius, I.H.M. van Stokkum, E. Zangrando, R.M. Williams, F. Würthner, Excited state interactions in calix[4]arene-perylene bisimide dye conjugates: Global and target analysis of supramolecular building blocks, Journal of Physical Chemistry C, 111 (2007) 13988-13996."

### Inspect data

In [ ]:
from pyglotaran_extras import plot_data_overview

oc_tol_data_path = "2016oc_tol.ascii"
plot_data_overview(oc_tol_data_path, nr_of_data_svd_vectors=5, linlog=True, linthresh=1);

## Global Analysis: sequential scheme with decay rates in decreasing order

### Used model and parameters

In [ ]:
from glotaran.utils.ipython import display_file

oc_tol_global_model_path = "models/oc_tol-global-model.yml"
oc_tol_global_parameters_path = "models/oc_tol-global-parameters.yml"

#### Model file

In [ ]:
display_file(oc_tol_global_model_path, syntax="yaml")

#### Parameters file

The free parameters are the center (the location of the maximum of the IRF at 550 nm, the dispersion_center) and the width of the IRF, and the kinetic parameters. The final state is long lived. The final decay rate has been fixed at a very small number to avoid numerical problems with the optimization.
The dispersion parameters have been fixed as well, but could have been estimated from the data.

Exercise 1: estimate the missing starting values for the parameters from the data inspection, and insert these in the models/oc_tol-global-parameters.yml file.

In [ ]:
display_file(oc_tol_global_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
from glotaran.optimization.optimize import optimize
from glotaran.project.scheme import Scheme

oc_tol_global_scheme = Scheme(
    model=oc_tol_global_model_path,
    parameters=oc_tol_global_parameters_path,
    maximum_number_function_evaluations=11,
    data={"oc_tol_data": oc_tol_data_path},
)
oc_tol_global_scheme.validate()

In [ ]:
oc_tol_global_result = optimize(oc_tol_global_scheme)

In [ ]:
oc_tol_global_result

In [ ]:
oc_tol_global_result.optimized_parameters

In [ ]:
from pyglotaran_extras.inspect import show_a_matrixes

show_a_matrixes(oc_tol_global_result)

### Result plots

#### Fit quality

In [ ]:
from pyglotaran_extras import plot_fitted_traces, select_plot_wavelengths

wavelengths = select_plot_wavelengths(oc_tol_global_result.data["oc_tol_data"])
plot_fitted_traces(oc_tol_global_result.data["oc_tol_data"], wavelengths, linlog=True, linthresh=1);

Note that in this plot the time axis is linear until 1 ps and logarithmic thereafter. This plot is more informative.

#### Overview

In [ ]:
from pyglotaran_extras import plot_overview

plot_overview(
    oc_tol_global_result.data["oc_tol_data"],
    linlog=True,
    nr_of_data_svd_vectors=4,
    nr_of_residual_svd_vectors=1,
);

## Coherent Artifact
The product of the matrix containing the zeroth, first and second derivative of the IRF multiplied by the IRF Associated Spectra (IRFAS) describes the Coherent Artefact.

In [ ]:
from pyglotaran_extras import plot_coherent_artifact

fig, axes = plot_coherent_artifact(
    oc_tol_global_result.data["oc_tol_data"], time_range=(-0.3, 0.3), figsize=(10, 4)
)
axes[0].set_xlabel("Time (ps)")
axes[0].get_legend().remove()
axes[1].set_xlabel("Wavelength (nm)")
axes[0].set_ylabel("")
axes[0].annotate("A", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
axes[1].annotate("B", xy=(0.02, 0.9), xycoords="axes fraction", fontsize=16)
fig.tight_layout()

## Dispersion curve

In [ ]:
oc_tol_global_result.data["oc_tol_data"].data.sel(time=slice(0, 2)).plot()
oc_tol_global_result.data["oc_tol_data"].irf_center_location.plot()

Exercise 2: Comment on the interpretation of the EADS, with the help of literature. Which EADS is the most difficult to interpret?

## First Target Analysis: small triplet yield

### Used model and parameters

In [ ]:
oc_tol_target_model_path = "models/oc_tol-target-model.yml"
oc_tol_target_parameters_path = "models/oc_tol-target-parameters.yml"

#### Model file

In [ ]:
display_file(oc_tol_target_model_path, syntax="yaml")

#### Parameters file

Exercise 3: In the regular sequential scheme (with decay rates in decreasing order) the final EADS was very small. This can be interpreted as the generation of a triplet with a small yield.
Insert the missing starting values in the models/oc_tol-target-parameters.yml file. Hint: use the above estimated kinetic and IRF parameters. Study the parameters file carefully. Try to find a better Tyield value, such that the bleaches near 530 nm of the 3rd and 4th SADS are comparable.

In [ ]:
display_file(oc_tol_target_parameters_path, syntax="yaml")

### Create scheme and optimize it

In [ ]:
oc_tol_target_scheme = Scheme(
    model=oc_tol_target_model_path,
    parameters=oc_tol_target_parameters_path,
    maximum_number_function_evaluations=1,
    data={"oc_tol_data": oc_tol_data_path},
)
oc_tol_target_scheme.validate()

In [ ]:
oc_tol_target_result = optimize(oc_tol_target_scheme)

In [ ]:
oc_tol_target_result

### Result plots

#### Fit quality

In [ ]:
fig_target, axes = plot_fitted_traces(
    oc_tol_target_result.data["oc_tol_data"], wavelengths, linlog=True, linthresh=1
);

#### Overview

In [ ]:
plot_overview(
    oc_tol_target_result.data["oc_tol_data"],
    figure_only=False,
    nr_of_data_svd_vectors=3,
    nr_of_residual_svd_vectors=1,
    linlog=True,
    linthresh=1,
);

Exercise 4: Comment on the interpretation of the SADS, with the help of literature. Which SADS is the most difficult to interpret?

In [ ]:
oc_tol_target_result.optimized_parameters

In [ ]:
compartments = oc_tol_target_scheme.model.initial_concentration["input1"].compartments

oc_tol_target_scheme.model.k_matrix["km1"].matrix_as_markdown(compartments).replace(
    "0.0000e+00", ""
)

In [ ]:
show_a_matrixes(oc_tol_target_result)